# Поиск аномалий

Методы обнаружения аномалий, как следует из названия, позволяют находить необычные объекты в выборке. Но что такое "необычные" и совпадает ли это определение у разных методов?

Начнём с поиска аномалий в текстах: научимся отличать вопросы о программировании от текстов из 20newsgroups про религию.

Подготовьте данные: в обучающую выборку возьмите 20 тысяч текстов из датасета Stack Overflow, а тестовую выборку сформируйте из 10 тысяч текстов со Stack Overflow и 100 текстов из класса soc.religion.christian датасета 20newsgroups (очень пригодится функция `fetch_20newsgroups(categories=['soc.religion.christian'])`). Тексты про программирование будем считать обычными, а тексты про религию — аномальными.

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

def read_stackoverflow_xml(file_path):
    # Чтение и парсинг XML файла
    print("Parsing file")
    tree = ET.parse(file_path)
    root = tree.getroot()
    print("Complete")
    posts = []
    
    # Проход по всем элементам row
    for row in root.findall('row')[:3:
        post_id = row.get('Id')
        post_type_id = row.get('PostTypeId')  # Тип поста: 1 - вопрос, 2 - ответ
        body = row.get('Body')
        if body:
            posts.append({'Id': post_id, 'PostTypeId': post_type_id, 'Body': body})
    
    # Конвертация в pandas DataFrame для удобной работы
    posts_df = pd.DataFrame(posts)
    return posts_df

# Чтение постов из XML файла
print("Starting reading file")

xml_file_path = 'posts.xml'  # путь к XML 
posts_df = read_stackoverflow_xml(xml_file_path)

# Выведем первые несколько строк
print(posts_df.head())

questions_df = posts_df[posts_df['PostTypeId'] == '1']


Starting reading file
Parsing file
Complete
  Id PostTypeId                                               Body
0  4          1  <p>I'm new to C# and I want to use a track-bar...
1  6          1  <p>I have an absolutely positioned <code>div</...
2  7          2  <p>An explicit cast to double isn't necessary....
3  8          1  <p>Are there any conversion tools for porting ...
4  9          1  <p>Given a <code>DateTime</code> representing ...


In [6]:
print(posts_df["Body"])

0        <p>I'm new to C# and I want to use a track-bar...
1        <p>I have an absolutely positioned <code>div</...
2        <p>An explicit cast to double isn't necessary....
3        <p>Are there any conversion tools for porting ...
4        <p>Given a <code>DateTime</code> representing ...
                               ...                        
29995    <p>But it's FUN! I mean it's more interesting ...
29996    <p>It depends on the number of images you are ...
29997    <p>I'm trying to find a simple way to change t...
29998    <p><strong><em><a href="http://dotnetnuke.mont...
29999    <p><a href="http://en.wikipedia.org/wiki/Casca...
Name: Body, Length: 30000, dtype: object


In [9]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
import pandas as pd
import random

# Выбираем 20 тысяч текстов для обучающей выборки
print("Train-Test StackOverflow")
train_df, test_df = train_test_split(posts_df["Body"], train_size=20000, random_state=42)

print("Religion Texts")
# Загружаем данные из 20newsgroups (категория: soc.religion.christian)
religion_texts = fetch_20newsgroups(categories=['soc.religion.christian'], remove=('headers', 'footers', 'quotes'))
religion_df = pd.DataFrame({'text': religion_texts.data, 'label': 1})  # label=1 для аномальных текстов

print("Labels")
# Метки для обычных текстов (label=0)
test_df['label'] = 0  # label=0 для обычных текстов

print("Test")
# Объединяем тексты Stack Overflow и религиозные тексты в тестовую выборку
test_df = pd.concat([test_df, religion_df.sample(100, random_state=42)], ignore_index=True)

print("Shuffle")
# Перемешиваем тестовую выборку для случайного распределения аномалий
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

Train-Test StackOverflow
Religion Texts
Labels
Test
Shuffle


In [10]:
print(religion_df)

                                                  text  label
0    I wrote in response to dlecoint@garnet.acns.fs...      1
1    A "new Christian" wrote that he was new to the...      1
2    : > \t   I'm a commited Christian that is batt...      1
3    My brother has been alienated from my parents ...      1
4       > [A very nice article on the DSS, which I ...      1
..                                                 ...    ...
594  Despite my trendy, liberal, feminist tendencie...      1
595  Where in the Bible is there *any* teaching abo...      1
596  \nI've sent the article.  In terms of the grou...      1
597  \n\tHmm...makes you wonder whether prayer "in ...      1
598  \n[deletia- and so on]\n\nI seem to have been ...      1

[599 rows x 2 columns]


In [11]:
posts_df["Body"].to_csv('StackOverflow_posts.csv', index=False)

**(1 балл)**

Проверьте качество выделения аномалий (precision и recall на тестовой выборке, если считать аномалии положительным классов, а обычные тексты — отрицательным) для IsolationForest. В качестве признаков используйте TF-IDF, где словарь и IDF строятся по обучающей выборке. Не забудьте подобрать гиперпараметры.

In [ ]:
#code here

**(5 баллов)**

Скорее всего, качество оказалось не на высоте. Разберитесь, в чём дело:
* посмотрите на тексты, которые выделяются как аномальные, а также на слова, соответствующие их ненулевым признакам
* изучите признаки аномальных текстов
* посмотрите на тексты из обучающей выборки, ближайшие к аномальным; действительно ли они похожи по признакам?

Сделайте выводы и придумайте, как избавиться от этих проблем. Предложите варианты двух типов: (1) в рамках этих же признаков (но которые, возможно, будут считаться по другим наборам данных) и методов и (2) без ограничений на изменения. Реализуйте эти варианты и проверьте их качество.

In [ ]:
#code here

### Эксперимент только с изменением датасета

In [ ]:
#code here

### Эксперимент с любыми изменениями

In [ ]:
#code here

Подготовьте выборку: удалите столбцы `['id', 'date', 'price', 'zipcode']`, сформируйте обучающую и тестовую выборки по 10 тысяч домов.

Добавьте в тестовую выборку 10 новых объектов, в каждом из которых испорчен ровно один признак — например, это может быть дом из другого полушария, из далёкого прошлого или будущего, с площадью в целый штат или с таким числом этажей, что самолётам неплохо бы его облетать стороной.

Посмотрим на методы обнаружения аномалий на более простых данных — уж на табличном датасете с 19 признаками всё должно работать как надо!

Скачайте данные о стоимости домов: https://www.kaggle.com/harlfoxem/housesalesprediction/data

In [ ]:
#code here

**Задание 9. (2 балла)**

Примените IsolationForest для поиска аномалий в этих данных, запишите их качество (как и раньше, это precision и recall). Проведите исследование:

Нарисуйте распределения всех признаков и обозначьте на этих распределениях объекты, которые признаны аномальными.

In [ ]:
#code here